In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

# =========================
# 1. Load site info (IGBP)
# =========================
sites = pd.read_csv('combined_sites_info.csv')
sites['SITE_ID'] = sites['SITE_ID'].str.strip()
sites['IGBP']    = sites['IGBP'].str.strip()
sites.rename(columns={'SITE_ID': 'station_id'}, inplace=True)

top4_ecos = ['ENF', 'GRA', 'CRO', 'DBF']

# ======================================
# 2. Load FLUXNET, ERA5, HoLAPS datasets
# ======================================
flux   = pd.read_excel('comprehensive_fluxnet_analysis.xlsx',
                       sheet_name='All_Selected_Periods_Data')
era5   = pd.read_excel('comprehensive_ERA5_analysis.xlsx',
                       sheet_name='All_Data')
holaps = pd.read_excel('comprehensive_holaps_analysis.xlsx',
                       sheet_name='All_Data')

flux['dataset']   = 'FLUXNET'
era5['dataset']   = 'ERA5'
holaps['dataset'] = 'HoLAPS'

# ====================================================
# 3. Derive canonical station_id (e.g. AT-Neu) in all
# ====================================================
def extract_station_id(name: str) -> str:
    """
    Extracts the FLUXNET-style site ID (e.g. AT-Neu) from
    station strings in all three products.
    """
    if isinstance(name, str):
        # look for pattern like XX-XXX (two letters, dash, 3 letters)
        m = re.search(r'[A-Z]{2}-[A-Za-z0-9]{3}', name)
        if m:
            return m.group(0)
    return name

for df in (flux, era5, holaps):
    df['station_id'] = df['station'].apply(extract_station_id)

# Check: this should now match SITE_ID values in sites [file:7]
# print(sorted(sites['station_id'].unique().tolist())[:10])
# print(sorted(flux['station_id'].unique().tolist())[:10])

# =====================================
# 4. Parameter mapping to common names
# =====================================

era5['parameter_std']   = era5['parameter']
holaps['parameter_std'] = holaps['parameter']

param_map_fluxnet = {
    'TA_F':          'Ta',
    'TS_F_MDS_1':    'Ts',
    'H_CORR':        'H',
    'LE_CORR':       'LE',
    'NETRAD':        'Rn',
    'P_F':           'rain',
    'SWC_F_MDS_1':   'sm1',
    'GPP_DT_VUT_MEAN': 'GPP',
    'G_F_MDS':         'G',
    'VPD_F':           'VPD',
}
flux['parameter_std'] = flux['parameter'].replace(param_map_fluxnet)

# ================================
# 5. Attach IGBP using station_id
# ================================
def add_igbp(df):
    return df.merge(sites[['station_id','IGBP']], on='station_id', how='left')

flux_eco   = add_igbp(flux)
era5_eco   = add_igbp(era5)
holaps_eco = add_igbp(holaps)

all_data = pd.concat([flux_eco, era5_eco, holaps_eco], ignore_index=True)

# Optional quick check:
# print('IGBP present in merged data:', sorted(all_data['IGBP'].dropna().unique()))

# ===============================
# 6. Common parameter set
# ===============================
common_params = ['Ta', 'Ts', 'H', 'LE', 'Rn', 'rain', 'sm1']

# ==========================================
# 7. Plot function: common plot per ecos/season
# ==========================================
output_dir = 'common_ecosystem_plots'
os.makedirs(output_dir, exist_ok=True)

def plot_common_by_ecosystem_season(data_all, ecos, season):
    df = data_all[(data_all['IGBP'] == ecos) & (data_all['season'] == season)]
    if df.empty:
        print(f'No data for {ecos} – {season}')
        return

    datasets   = ['FLUXNET', 'HoLAPS', 'ERA5']
    sig_levels = ['insig', 'medium', 'high']

    counts = {(p, d, s): 0 for p in common_params for d in datasets for s in sig_levels}

    for _, row in df.iterrows():
        p = row['parameter_std']
        d = row['dataset']
        s = row['significance']
        if p in common_params and d in datasets and s in sig_levels:
            counts[(p, d, s)] += 1

    x = np.arange(len(common_params))
    group_width = 0.8
    bar_width   = group_width / len(datasets)

    fig, ax = plt.subplots(figsize=(18, 8))

    for j, d in enumerate(datasets):
        x_shift = x - group_width/2 + bar_width/2 + j*bar_width
        insig = np.array([counts[(p, d, 'insig')]  for p in common_params])
        med   = np.array([counts[(p, d, 'medium')] for p in common_params])
        high  = np.array([counts[(p, d, 'high')]   for p in common_params])

        ax.bar(x_shift, insig, bar_width,
               color='#d62728', edgecolor='black', linewidth=0.8, alpha=0.8)
        ax.bar(x_shift, med, bar_width, bottom=insig,
               color='#ff7f0e', edgecolor='black', linewidth=0.8, alpha=0.8)
        ax.bar(x_shift, high, bar_width, bottom=insig+med,
               color='#2ca02c', edgecolor='black', linewidth=0.8, alpha=0.8)

    ax.set_xticks(x)
    ax.set_xticklabels(common_params, rotation=45, ha='right', fontsize=11)
    ax.set_ylabel('Number of stations', fontsize=13, fontweight='bold')

    season_label = 'Yearly' if season == 'yearly' else 'JJA (Summer)'
    ax.set_title(f'{ecos} – {season_label} – FLUXNET vs HOLAPS vs ERA5',
                 fontsize=15, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    import matplotlib.ticker as mticker
    ax.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))


    from matplotlib.patches import Patch
    legend_handles = [
        Patch(facecolor='#d62728', edgecolor='black', label='Not Significant (p ≥ 0.2)'),
        Patch(facecolor='#ff7f0e', edgecolor='black', label='Moderately Significant (0.1 ≤ p < 0.2)'),
        Patch(facecolor='#2ca02c', edgecolor='black', label='Highly Significant (p < 0.1)'),
    ]
    ax.legend(handles=legend_handles, loc='upper right', fontsize=10, title='Significance')

    plt.tight_layout()
    fname = f'COMMON_{ecos}_{season}.png'
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()
    print('Saved', fname)

for ecos in top4_ecos:
    for season in ['yearly', 'JJA']:
        plot_common_by_ecosystem_season(all_data, ecos, season)


Saved COMMON_ENF_yearly.png
Saved COMMON_ENF_JJA.png
Saved COMMON_GRA_yearly.png
Saved COMMON_GRA_JJA.png
Saved COMMON_CRO_yearly.png
Saved COMMON_CRO_JJA.png
Saved COMMON_DBF_yearly.png
Saved COMMON_DBF_JJA.png


In [6]:
flux_eco

,station,parameter,season,significance,p_value,slope,score_contribution,period,dataset,station_id,parameter_std,IGBP
0,progressive_trend_tables_AT-Neu,LE_CORR,yearly,insig,0.533417,0.368422,0.1,2002-2012 (11),FLUXNET,AT-Neu,LE,GRA
1,progressive_trend_tables_AT-Neu,H_CORR,yearly,insig,0.275758,0.203130,0.1,2002-2012 (11),FLUXNET,AT-Neu,H,GRA
2,progressive_trend_tables_AT-Neu,GPP_DT_VUT_MEAN,yearly,insig,0.755497,0.015760,0.1,2002-2012 (11),FLUXNET,AT-Neu,GPP,GRA
3,progressive_trend_tables_AT-Neu,LE_CORR,JJA,insig,0.640429,0.488392,0.1,2002-2012 (11),FLUXNET,AT-Neu,LE,GRA
4,progressive_trend_tables_AT-Neu,H_CORR,JJA,medium,0.161125,0.544858,0.5,2002-2012 (11),FLUXNET,AT-Neu,H,GRA
...,...,...,...,...,...,...,...,...,...,...,...,...
590,progressive_trend_tables_SE-Nor,LE_CORR,JJA,insig,0.640429,-0.464687,0.1,2014-2024 (11),FLUXNET,SE-Nor,LE,ENF
591,progressive_trend_tables_SE-Nor,H_CORR,JJA,insig,0.436275,-0.505537,0.1,2014-2024 (11),FLUXNET,SE-Nor,H,ENF
592,progressive_trend_tables_SE-Svb,LE_CORR,JJA,insig,0.755497,0.075807,0.1,2014-2024 (11),FLUXNET,SE-Svb,LE,ENF
593,progressive_trend_tables_SE-Svb,H_CORR,JJA,medium,0.161125,-1.104367,0.5,2014-2024 (11),FLUXNET,SE-Svb,H,ENF


In [7]:
era5_eco

,station,parameter,season,significance,p_value,slope,period,dataset,station_id,parameter_std,IGBP
0,era5_trend_tables_AT-Neu,GHF,yearly,insig,0.922462,1206.023517,2001-2020 (20),ERA5,AT-Neu,GHF,GRA
1,era5_trend_tables_AT-Neu,H,yearly,insig,0.256145,-4523.413088,2001-2020 (20),ERA5,AT-Neu,H,GRA
2,era5_trend_tables_AT-Neu,LE,yearly,insig,0.205754,10569.517686,2001-2020 (20),ERA5,AT-Neu,LE,GRA
3,era5_trend_tables_AT-Neu,Rn,yearly,insig,0.673189,3617.628039,2001-2020 (20),ERA5,AT-Neu,Rn,GRA
4,era5_trend_tables_AT-Neu,Ta,yearly,high,0.021248,0.059618,2001-2020 (20),ERA5,AT-Neu,Ta,GRA
...,...,...,...,...,...,...,...,...,...,...,...
1051,era5_trend_tables_SE-Htm,rain,JJA,medium,0.183447,-0.033039,2001-2020 (20),ERA5,SE-Htm,rain,ENF
1052,era5_trend_tables_SE-Htm,sm1,JJA,high,0.040955,-0.002102,2001-2020 (20),ERA5,SE-Htm,sm1,ENF
1053,era5_trend_tables_SE-Htm,sm2,JJA,high,0.014961,-0.003323,2001-2020 (20),ERA5,SE-Htm,sm2,ENF
1054,era5_trend_tables_SE-Htm,sm3,JJA,high,0.021248,-0.002205,2001-2020 (20),ERA5,SE-Htm,sm3,ENF


In [3]:
import pandas as pd
import re

# =========================
# 1. Load site info (IGBP)
# =========================
sites = pd.read_csv('combined_sites_info.csv')
sites['SITE_ID'] = sites['SITE_ID'].str.strip()
sites['IGBP']    = sites['IGBP'].str.strip()
sites.rename(columns={'SITE_ID': 'station_id'}, inplace=True)

# ======================================
# 2. Load FLUXNET, ERA5, HoLAPS datasets
# ======================================
flux   = pd.read_excel('comprehensive_fluxnet_analysis.xlsx',
                       sheet_name='All_Selected_Periods_Data')
era5   = pd.read_excel('comprehensive_ERA5_analysis.xlsx',
                       sheet_name='All_Data')
holaps = pd.read_excel('comprehensive_holaps_analysis.xlsx',
                       sheet_name='All_Data')

flux['dataset']   = 'FLUXNET'
era5['dataset']   = 'ERA5'
holaps['dataset'] = 'HoLAPS'

# ====================================================
# 3. Derive canonical station_id (e.g. AT-Neu) in all
# ====================================================
def extract_station_id(name: str) -> str:
    """
    Extracts the FLUXNET-style site ID (e.g. AT-Neu) from
    station strings in all three products.
    """
    if isinstance(name, str):
        # look for pattern like XX-XXX (two letters, dash, 3 letters)
        m = re.search(r'[A-Z]{2}-[A-Za-z0-9]{3}', name)
        if m:
            return m.group(0)
    return name

for df in (flux, era5, holaps):
    df['station_id'] = df['station'].apply(extract_station_id)

# =====================================
# 4. Parameter mapping to common names
# =====================================
era5['parameter_std']   = era5['parameter']
holaps['parameter_std'] = holaps['parameter']

param_map_fluxnet = {
    'TA_F':          'Ta',
    'TS_F_MDS_1':    'Ts',
    'H_CORR':        'H',
    'LE_CORR':       'LE',
    'NETRAD':        'Rn',
    'P_F':           'rain',
    'SWC_F_MDS_1':   'sm1',
    'GPP_DT_VUT_MEAN': 'GPP',
    'G_F_MDS':         'G',
    'VPD_F':           'VPD',
}
flux['parameter_std'] = flux['parameter'].replace(param_map_fluxnet)

# ================================
# 5. Attach IGBP using station_id
# ================================
def add_igbp(df):
    return df.merge(sites[['station_id','IGBP']], on='station_id', how='left')

flux_eco   = add_igbp(flux)
era5_eco   = add_igbp(era5)
holaps_eco = add_igbp(holaps)

# ================================================================
# 6. Function to find exclusive stations by season
# ================================================================
def find_exclusive_stations_by_season(season_name):
    """Find exclusive FLUXNET stations for LE_CORR and H_CORR for a specific season"""
    print(f"\n{'='*80}")
    print(f"ANALYSIS FOR {season_name.upper()} SEASON")
    print(f"{'='*80}")
    
    # Filter data by season
    flux_season = flux_eco[flux_eco['season'] == season_name]
    era5_season = era5_eco[era5_eco['season'] == season_name]
    holaps_season = holaps_eco[holaps_eco['season'] == season_name]
    
    # Filter for LE_CORR and H_CORR in FLUXNET
    flux_le = flux_season[flux_season['parameter'] == 'LE_CORR']
    flux_h = flux_season[flux_season['parameter'] == 'H_CORR']
    
    # Get unique stations for each parameter in FLUXNET
    flux_le_stations = set(flux_le['station_id'].unique())
    flux_h_stations = set(flux_h['station_id'].unique())
    
    # Get stations from other datasets that have LE or H parameters
    era5_le_stations = set(era5_season[era5_season['parameter_std'] == 'LE']['station_id'].unique())
    era5_h_stations = set(era5_season[era5_season['parameter_std'] == 'H']['station_id'].unique())
    
    holaps_le_stations = set(holaps_season[holaps_season['parameter_std'] == 'LE']['station_id'].unique())
    holaps_h_stations = set(holaps_season[holaps_season['parameter_std'] == 'H']['station_id'].unique())
    
    print(f"\n=== FLUXNET STATIONS WITH LE_CORR ({season_name}) ===")
    print(f"Total: {len(flux_le_stations)} stations")
    
    print(f"\n=== FLUXNET STATIONS WITH H_CORR ({season_name}) ===")
    print(f"Total: {len(flux_h_stations)} stations")
    
    # Find exclusive stations (in FLUXNET but not in ERA5 or HoLAPS)
    exclusive_le_stations = flux_le_stations - (era5_le_stations | holaps_le_stations)
    exclusive_h_stations = flux_h_stations - (era5_h_stations | holaps_h_stations)
    
    print(f"\n=== STATIONS EXCLUSIVE TO FLUXNET FOR LE_CORR ({season_name}) ===")
    print(f"Total exclusive stations: {len(exclusive_le_stations)}")
    
    exclusive_le_details = []
    if exclusive_le_stations:
        for station in sorted(exclusive_le_stations):
            # Get ecosystem and significance
            station_data = flux_le[flux_le['station_id'] == station]
            ecosystem = sites[sites['station_id'] == station]['IGBP'].values
            eco_str = ecosystem[0] if len(ecosystem) > 0 else "Unknown"
            
            sig_counts = station_data['significance'].value_counts().to_dict()
            sig_str = ', '.join([f"{k}: {v}" for k, v in sig_counts.items()])
            
            exclusive_le_details.append({
                'station': station,
                'ecosystem': eco_str,
                'significance': sig_str
            })
            
            print(f"  - {station} (Ecosystem: {eco_str}, Significance: {sig_str})")
    else:
        print("  No exclusive stations found")
    
    print(f"\n=== STATIONS EXCLUSIVE TO FLUXNET FOR H_CORR ({season_name}) ===")
    print(f"Total exclusive stations: {len(exclusive_h_stations)}")
    
    exclusive_h_details = []
    if exclusive_h_stations:
        for station in sorted(exclusive_h_stations):
            # Get ecosystem and significance
            station_data = flux_h[flux_h['station_id'] == station]
            ecosystem = sites[sites['station_id'] == station]['IGBP'].values
            eco_str = ecosystem[0] if len(ecosystem) > 0 else "Unknown"
            
            sig_counts = station_data['significance'].value_counts().to_dict()
            sig_str = ', '.join([f"{k}: {v}" for k, v in sig_counts.items()])
            
            exclusive_h_details.append({
                'station': station,
                'ecosystem': eco_str,
                'significance': sig_str
            })
            
            print(f"  - {station} (Ecosystem: {eco_str}, Significance: {sig_str})")
    else:
        print("  No exclusive stations found")
    
    # Return results for saving
    return {
        'season': season_name,
        'exclusive_le': exclusive_le_details,
        'exclusive_h': exclusive_h_details,
        'flux_le_stations': list(flux_le_stations),
        'flux_h_stations': list(flux_h_stations),
        'era5_le_stations': list(era5_le_stations),
        'era5_h_stations': list(era5_h_stations),
        'holaps_le_stations': list(holaps_le_stations),
        'holaps_h_stations': list(holaps_h_stations)
    }

# ================================================================
# 7. Run analysis for both seasons
# ================================================================
print("=" * 80)
print("EXCLUSIVE FLUXNET STATIONS ANALYSIS FOR LE_CORR AND H_CORR")
print("=" * 80)

# Analyze both seasons
jja_results = find_exclusive_stations_by_season('JJA')
yearly_results = find_exclusive_stations_by_season('yearly')

# ================================================================
# 8. Create summary DataFrames and save to CSV
# ================================================================
def create_summary_dataframe(results, season_name):
    """Create summary DataFrame for a specific season"""
    summary_data = []
    
    # Get all unique stations across all datasets for this season
    all_stations = set()
    all_stations.update(results['flux_le_stations'])
    all_stations.update(results['flux_h_stations'])
    all_stations.update(results['era5_le_stations'])
    all_stations.update(results['era5_h_stations'])
    all_stations.update(results['holaps_le_stations'])
    all_stations.update(results['holaps_h_stations'])
    
    for station in sorted(all_stations):
        # Get ecosystem
        eco_data = sites[sites['station_id'] == station]
        ecosystem = eco_data['IGBP'].values[0] if len(eco_data) > 0 else "Unknown"
        
        # Check LE parameter
        in_fluxnet_le = station in results['flux_le_stations']
        in_era5_le = station in results['era5_le_stations']
        in_holaps_le = station in results['holaps_le_stations']
        
        # Check H parameter
        in_fluxnet_h = station in results['flux_h_stations']
        in_era5_h = station in results['era5_h_stations']
        in_holaps_h = station in results['holaps_h_stations']
        
        # Check if exclusive
        exclusive_le = station in [item['station'] for item in results['exclusive_le']]
        exclusive_h = station in [item['station'] for item in results['exclusive_h']]
        
        summary_data.append({
            'Station': station,
            'Ecosystem': ecosystem,
            'Season': season_name,
            'FLUXNET_LE': 'Yes' if in_fluxnet_le else 'No',
            'ERA5_LE': 'Yes' if in_era5_le else 'No',
            'HoLAPS_LE': 'Yes' if in_holaps_le else 'No',
            'Exclusive_FLUXNET_LE': 'Yes' if exclusive_le else 'No',
            'FLUXNET_H': 'Yes' if in_fluxnet_h else 'No',
            'ERA5_H': 'Yes' if in_era5_h else 'No',
            'HoLAPS_H': 'Yes' if in_holaps_h else 'No',
            'Exclusive_FLUXNET_H': 'Yes' if exclusive_h else 'No'
        })
    
    return pd.DataFrame(summary_data)

# Create DataFrames for both seasons
jja_summary_df = create_summary_dataframe(jja_results, 'JJA')
yearly_summary_df = create_summary_dataframe(yearly_results, 'Yearly')

# Combine into one DataFrame
combined_summary = pd.concat([jja_summary_df, yearly_summary_df], ignore_index=True)

# ================================================================
# 9. Save results to Excel with multiple sheets
# ================================================================
output_file = 'exclusive_fluxnet_stations_analysis.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Save combined summary
    combined_summary.to_excel(writer, sheet_name='All_Seasons_Summary', index=False)
    
    # Save separate season summaries
    jja_summary_df.to_excel(writer, sheet_name='JJA_Season', index=False)
    yearly_summary_df.to_excel(writer, sheet_name='Yearly_Season', index=False)
    
    # Create detailed exclusive stations sheets
    exclusive_le_jja = pd.DataFrame(jja_results['exclusive_le'])
    exclusive_h_jja = pd.DataFrame(jja_results['exclusive_h'])
    exclusive_le_yearly = pd.DataFrame(yearly_results['exclusive_le'])
    exclusive_h_yearly = pd.DataFrame(yearly_results['exclusive_h'])
    
    if not exclusive_le_jja.empty:
        exclusive_le_jja.to_excel(writer, sheet_name='Exclusive_LE_JJA', index=False)
    if not exclusive_h_jja.empty:
        exclusive_h_jja.to_excel(writer, sheet_name='Exclusive_H_JJA', index=False)
    if not exclusive_le_yearly.empty:
        exclusive_le_yearly.to_excel(writer, sheet_name='Exclusive_LE_Yearly', index=False)
    if not exclusive_h_yearly.empty:
        exclusive_h_yearly.to_excel(writer, sheet_name='Exclusive_H_Yearly', index=False)

print(f"\n{'='*80}")
print(f"Results saved to: {output_file}")
print(f"{'='*80}")

# Print quick summary
print("\n=== QUICK SUMMARY ===")
print(f"JJA Season:")
print(f"  - Exclusive LE_CORR stations: {len(jja_results['exclusive_le'])}")
print(f"  - Exclusive H_CORR stations: {len(jja_results['exclusive_h'])}")
print(f"  - Total FLUXNET stations with LE_CORR: {len(jja_results['flux_le_stations'])}")
print(f"  - Total FLUXNET stations with H_CORR: {len(jja_results['flux_h_stations'])}")

print(f"\nYearly Season:")
print(f"  - Exclusive LE_CORR stations: {len(yearly_results['exclusive_le'])}")
print(f"  - Exclusive H_CORR stations: {len(yearly_results['exclusive_h'])}")
print(f"  - Total FLUXNET stations with LE_CORR: {len(yearly_results['flux_le_stations'])}")
print(f"  - Total FLUXNET stations with H_CORR: {len(yearly_results['flux_h_stations'])}")

print(f"\nExcel file contains {len(combined_summary)} station entries across both seasons.")

EXCLUSIVE FLUXNET STATIONS ANALYSIS FOR LE_CORR AND H_CORR

ANALYSIS FOR JJA SEASON

=== FLUXNET STATIONS WITH LE_CORR (JJA) ===
Total: 50 stations

=== FLUXNET STATIONS WITH H_CORR (JJA) ===
Total: 50 stations

=== STATIONS EXCLUSIVE TO FLUXNET FOR LE_CORR (JJA) ===
Total exclusive stations: 7
  - FI-Hyy (Ecosystem: ENF, Significance: high: 1)
  - FI-Let (Ecosystem: ENF, Significance: insig: 1)
  - FI-Sod (Ecosystem: ENF, Significance: high: 1)
  - IL-Yat (Ecosystem: Unknown, Significance: high: 1)
  - SE-Deg (Ecosystem: GRA, Significance: medium: 1)
  - SE-Nor (Ecosystem: ENF, Significance: insig: 1)
  - SE-Svb (Ecosystem: ENF, Significance: insig: 1)

=== STATIONS EXCLUSIVE TO FLUXNET FOR H_CORR (JJA) ===
Total exclusive stations: 7
  - FI-Hyy (Ecosystem: ENF, Significance: insig: 1)
  - FI-Let (Ecosystem: ENF, Significance: insig: 1)
  - FI-Sod (Ecosystem: ENF, Significance: insig: 1)
  - IL-Yat (Ecosystem: Unknown, Significance: insig: 1)
  - SE-Deg (Ecosystem: GRA, Significance: 